In [14]:
%load_ext autoreload
%autoreload 2

from gptchem.data import get_photoswitch_data
from loguru import logger
logger.enable("gptchem")
from gptchem.extractor import RegressionExtractor
from gptchem.formatter import RegressionFormatter
from gptchem.tuner import Tuner 
from gptchem.querier import Querier

from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = get_photoswitch_data()

In [6]:
formatter = RegressionFormatter(
    representation_column="SMILES",
    label_column="E isomer pi-pi* wavelength in nm",
    property_name="E isomer pi-pi* wavelength in nm",
)

In [7]:
formatted = formatter(data)

In [8]:
formatted

,prompt,completion,label,representation
0,What is the E isomer pi-pi* wavelength in nm o...,310.0@@@,310.0,C[N]1C=CC(=N1)N=NC2=CC=CC=C2
1,What is the E isomer pi-pi* wavelength in nm o...,310.0@@@,310.0,C[N]1C=NC(=N1)N=NC2=CC=CC=C2
2,What is the E isomer pi-pi* wavelength in nm o...,325.0@@@,325.0,C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2
3,What is the E isomer pi-pi* wavelength in nm o...,328.0@@@,328.0,C[N]1C=C(C=N1)N=NC2=CC=CC=C2
4,What is the E isomer pi-pi* wavelength in nm o...,335.0@@@,335.0,C[N]1N=C(C)C(=C1C)N=NC2=CC=CC=C2
...,...,...,...,...
385,What is the E isomer pi-pi* wavelength in nm o...,456.0@@@,456.0,OC%38=C%39N=CC=CC%39=C(/N=N/C%40=NC%41=CC(C)=C...
386,What is the E isomer pi-pi* wavelength in nm o...,437.0@@@,437.0,OC%42=C%43N=CC=CC%43=C(/N=N/C%44=NC%45=CC=CC=C...
387,What is the E isomer pi-pi* wavelength in nm o...,545.0@@@,545.0,N#CC1C(SC(/N=N/C2=NC(C=CC([N+]([O-])=O)=C3)=C3...
388,What is the E isomer pi-pi* wavelength in nm o...,535.0@@@,535.0,N#Cc5c(c6ccc(Cl)cc6)c(/N=N/C7=NC(C=CC([N+]([O-...


In [9]:
train, test = train_test_split(formatted, test_size=0.2)

In [12]:
train_subset, val = train_test_split(train, train_size=0.2)

In [15]:
tuner = Tuner()

In [16]:
tune_res = tuner.tune(train_subset)

Upload progress: 100%|██████████| 13.3k/13.3k [00:00<00:00, 7.86Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/photoswitch/out/20230117_232713/train.jsonl: file-Cx3yXvcQ3jSYKmfy9KndQ6ee


2023-01-17 23:27:24.709 | DEBUG    | gptchem.tuner:tune:186 - Requested fine tuning. {
  "created_at": 1673994444,
  "events": [
    {
      "created_at": 1673994444,
      "level": "info",
      "message": "Created fine-tune: ft-9H4zpmFR59Q4xTlWhNM6lPa9",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-9H4zpmFR59Q4xTlWhNM6lPa9",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 13282,
      "created_at": 1673994444,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/photoswitch/out/20230117_232713/train.jsonl",
      "id": "file-Cx3yXvcQ3jSYKmfy9KndQ6ee",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
     

Fine-tune ft-PpWA3b7JADC4wTddEvVJP4VH has the status "pending" and will not be logged
🎉 wandb sync completed successfully


In [17]:
querier = Querier(tune_res['model_name'])

In [18]:
completions = querier(test)